## Memory Component

After receiving the initial user input and before executing the core logic, the chain reads the history from the memory component to enhance the user input. After executing the core logic and before returning the answer, the chain writes the current interaction's input and output into memory to update the conversation history.

### ConversationBufferMemory

`ConversationBufferMemory` is a memory component class provided by LangChain. It accurately records conversation history messages in a list. However, as interactions increase and the number of characters grows, the character count of the conversation history may cause the token count of the enhanced prompt to exceed the context limit, eventually leading to model call failure.

In [3]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context(
    {"input": "Hi! I am Johnson."},
    {"output": "Hi, Johnson!"}
    )

In [4]:
memory.chat_memory.messages

[HumanMessage(content='I am Johnson.'), AIMessage(content='Hi, Johnson!')]

In [5]:
memory.load_memory_variables({})

{'history': 'Human: I am Johnson.\nAI: Hi, Johnson!'}

## ConversationBufferWindowMemory

`ConversationBufferWindowMemory` continuously records conversation history but only uses the most recent K interactions. This sliding window mechanism ensures that the cache size does not become too large.

In [6]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=5)
memory.save_context(
    {"input": "Hi! I am Johnson."},
    {"output":"Hi, Johnson!"}
) 
memory.save_context(
    {"input": "I am from Taiwan"},
    {"output":"Wow! Taiwan is a great country!"}
) 

In [7]:
memory.load_memory_variables({})

{'history': 'Human: Hi! I am Johnson.\nAI: Hi, Johnson!\nHuman: I am from Taiwan\nAI: Wow! Taiwan is a great country!'}

In [8]:
memory.chat_memory.messages

[HumanMessage(content='Hi! I am Johnson.'),
 AIMessage(content='Hi, Johnson!'),
 HumanMessage(content='I am from Taiwan'),
 AIMessage(content='Wow! Taiwan is a great country!')]

### CoversationSummaryMemory

`ConversationSummaryMemory` is a slightly more complex type of memory. This memory summarizes the content of the conversation over time and stores the current summary in memory. It then injects the conversation summary into the prompt or chain when needed. `ConversationSummaryMemory` is useful for longer conversation interactions because including past history verbatim in the prompt would consume too many tokens.

In [9]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(temperature=0)
memory = ConversationSummaryMemory(llm=chat_model)
memory.save_context(
    {"input": "Hi! I am Johnson."},
    {"output":"Hi, Johnson!"}
) 
memory.save_context(
    {"input": "I am from Taiwan"},
    {"output":"Wow! Taiwan is a great country!"}
) 
memory.save_context(
    {"input": "Tell me what you know about Taiwan."},
    {"output":"Taiwan, an island nation off China's coast, has a vibrant democracy, strong economy, and complex political status with China."}
)


In [10]:
memory.load_memory_variables({})

{'history': "Johnson introduces himself as being from Taiwan, to which the AI responds with enthusiasm about the country. The AI shares that Taiwan is an island nation off China's coast with a vibrant democracy, strong economy, and complex political status with China."}

The following image illustrates the token overhead trend of conversation history information generated by different types of memory components as conversation interactions increase.

![images](docs/memory.png)